# Graphing model predictions against actual values

In [1]:
import torch
import os
import json

import numpy as np
import pandas as pd

import preproc

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## read the dataset

In [3]:
data_dir = 'dataset/preprocessed'
train = np.load(os.path.join(data_dir, 'train.npy'))
valid = np.load(os.path.join(data_dir, 'valid.npy'))
test = np.load(os.path.join(data_dir, 'test.npy'))
# load the bounds as well
bounds_path = os.path.join(data_dir, 'bounds.json')
bounds = {}
with open(bounds_path)as file:
    bounds = json.load(file)

cols = list(bounds.keys())

# split into inputs and labels
x_train = torch.Tensor(train[:,:64,:])
y_train = torch.Tensor(train[:,64,:])

x_valid = torch.Tensor(valid[:,:64,:])
y_valid = torch.Tensor(valid[:,64,:])

x_test = torch.Tensor(test[:,:64,:])
y_test = torch.Tensor(test[:,64,:])

## Load the model

In [4]:
from lstm_model import FirePredictor

hidden_dim = 1024
n_layers = 4
model = FirePredictor(train.shape[-1], preproc.WINDOW_SIZE, hidden_dim, n_layers)
model.eval()
model.from_json('models/single_lstm.json')

## Run the model

In [15]:
with torch.no_grad():
    p_valid = model(x_valid.to(device))
    p_train = model(x_train[0:x_valid.shape[0]].to(device))

## un-preprocess the results

In [16]:
train_p_frame = pd.DataFrame(p_train.cpu().detach().numpy(), columns=cols)
train_p_frame = preproc.unprocess(train_p_frame, bounds)
display(train_p_frame.shape)
display(train_p_frame.head())

train_y_frame = pd.DataFrame(y_train.cpu().detach().numpy(), columns=cols)
train_y_frame = preproc.unprocess(train_y_frame, bounds)
display(train_y_frame.shape)
display(train_y_frame.head())

(378, 3)

,latitude,longitude,timestamp
0,-15.190112,132.504479,1.564718e+09
1,-18.898217,136.315598,1.564731e+09
2,-19.677320,137.136176,1.564719e+09
3,-29.259334,148.228384,1.564713e+09
4,-25.048634,145.339336,1.564695e+09


(1135, 3)

,latitude,longitude,timestamp
0,-16.154501,136.711701,1.564646e+09
1,-14.685501,126.769900,1.564652e+09
2,-34.146500,116.388100,1.564652e+09
3,-30.340300,147.721001,1.564656e+09
4,-19.510200,146.422000,1.564656e+09


In [17]:
valid_p_frame = pd.DataFrame(p_valid.cpu().detach().numpy(), columns=cols)
valid_p_frame = preproc.unprocess(valid_p_frame, bounds)
display(valid_p_frame.shape)
display(valid_p_frame.head())

valid_y_frame = pd.DataFrame(y_valid.detach().numpy(), columns=cols)
valid_y_frame = preproc.unprocess(valid_y_frame, bounds)
display(valid_y_frame.shape)
display(valid_y_frame.head())

(378, 3)

,latitude,longitude,timestamp
0,-13.764012,135.355106,1.567947e+09
1,-14.058787,135.371191,1.567958e+09
2,-13.799219,135.100542,1.567950e+09
3,-13.488889,134.703219,1.567955e+09
4,-13.557406,134.716156,1.567951e+09


(378, 3)

,latitude,longitude,timestamp
0,-13.254899,130.859100,1.567943e+09
1,-12.905000,136.309600,1.567943e+09
2,-13.999600,130.766100,1.567943e+09
3,-14.113900,130.753400,1.567943e+09
4,-12.250000,134.777999,1.567943e+09


In [18]:
display(valid_y_frame.std(axis=0))
display(valid_p_frame.std(axis=0))

latitude          7.502988
longitude        10.088694
timestamp    171899.076166
dtype: float64

latitude          5.600824
longitude         6.966525
timestamp    166135.033064
dtype: float64

# Make the graph

In [9]:
import folium
import geojson
from datetime import datetime

Convert data to TimestampedGeoJSON

In [19]:
def get_points(df):
    points = list(zip(df['longitude'], df['latitude']))
    return geojson.MultiPoint(points)

train_y_markers = get_points(train_y_frame)
train_p_markers = get_points(train_p_frame)
valid_y_markers = get_points(valid_y_frame)
valid_p_markers = get_points(valid_p_frame)

In [24]:
m = folium.Map(
    [-30.0, 140.0],
    zoom_start=4,
    tiles='OpenTopoMap'
)

# add training y markers to the map
folium.GeoJson(
    train_y_markers,
    name="Predicted Pixels",
    marker=folium.Circle(radius=100, fill_color = 'blue', fill_opacity=0.5, color="black", weight=1),
    style_function=lambda x: {
        "fillColor": 'blue',
        'radius': 20000
    },
    highlight_function=lambda x: {"fillOpacity": 0.8}
).add_to(m)

# add training predicted markers to the map
folium.GeoJson(
    train_p_markers,
    name="Predicted Pixels",
    marker=folium.Circle(radius=100, fill_color = 'green', fill_opacity=0.5, color="black", weight=1),
    style_function=lambda x: {
        "fillColor": 'green',
        'radius': 20000
    },
    highlight_function=lambda x: {"fillOpacity": 0.8}
).add_to(m)

# add validation y markers to the map
folium.GeoJson(
    valid_y_markers,
    name="Fire Pixels",
    marker=folium.Circle(radius=100, fill_color = 'red', fill_opacity=0.5, color="black", weight=1),
    style_function=lambda x: {
        "fillColor": 'red',
        'radius': 5000
    },
    highlight_function=lambda x: {"fillOpacity": 0.8}
).add_to(m)

# add validation predicted markers to the map
folium.GeoJson(
    valid_p_markers,
    name="Predicted Pixels",
    marker=folium.Circle(radius=100, fill_color = 'orange', fill_opacity=0.5, color="black", weight=1),
    style_function=lambda x: {
        "fillColor": 'orange',
        'radius': 5000
    },
    highlight_function=lambda x: {"fillOpacity": 0.8}
).add_to(m)
m